Facciamo il training del modello che crea le maschere e verifichiamo se il tempo di inferenza si riduce.

In [ ]:
from utils import load_and_prepare_data
from neural_network import NeuralNetwork
from masknet import *
import torch
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time


In [ ]:
numerical_cols = [
        "duration",
        "dst_bytes",
        "missed_bytes",
        "src_bytes",
        "src_ip_bytes",
        "src_pkts",
        "dst_pkts",
        "dst_ip_bytes",
        "http_request_body_len",
        "http_response_body_len"

    ]

categorical_cols = [
        "proto",
        "conn_state",
        "http_status_code",
        "http_method",
        "http_orig_mime_types",
        "http_resp_mime_types",
    ]


target_col = 'type'
values_to_remove = {'type': ['mitm', 'dos']}

dataset_path = './Dataset/http_ton.csv'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MASK_THRESHOLD = 0.3


In [ ]:
best_model = NeuralNetwork.load('best_model.pt', device=device)
config = best_model.config

In [ ]:
mask_generator = MaskGenerator(
    cat_cardinalities=config['cat_cardinalities'],
    embedding_dims=config['embedding_dims'],
    num_numerical_features=config['num_numerical_features'],
    mask_sizes=config['hidden_layers_sizes'],
    share_embeddings=best_model.embeddings
)

In [ ]:
train_dataloader, valid_dataloader, test_dataloader, cat_cardinalities, cw, target_names = load_and_prepare_data(
    file_path=dataset_path,
    target_col=target_col,
    numerical_cols=numerical_cols,
    categorical_cols=categorical_cols,
    rows_to_remove=values_to_remove,
    batch_size=4096
)

In [ ]:
mask_generator.fit(
    model=best_model,
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    device=device,
    threshold=MASK_THRESHOLD,
    alpha=0.0001,
    epochs=20,
    warmup_epochs=5,
    lr=0.001
)

### Senza maschere:

In [ ]:
y_pred = best_model.predict(test_dataloader, device)
y_true = torch.cat([y for _, _, y in test_dataloader]).numpy()
print("\n=== Classification Report SENZA MASCHERE ===")
print(classification_report(y_true, y_pred.numpy(), target_names=target_names, digits=4))

### Con maschere

In [ ]:
y_pred = best_model.predict_with_masks(test_dataloader, device, mask_generator, threshold=MASK_THRESHOLD)
y_true = torch.cat([y for _, _, y in test_dataloader]).numpy()
print("\n=== Classification Report CON MASCHERE ===")
print(classification_report(y_true, y_pred.numpy(), target_names=target_names, digits=4))

### Benchmark Temporale

In [ ]:
benchmark_inference_speed(
    mask_generator=mask_generator,
    dataloader=test_dataloader,
    device=device,
    threshold=MASK_THRESHOLD,
    num_runs=10
)